In [1]:
def harris_corner_detector(img):
 
 gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
 # apply sobel filter in to get gradient image
 gray_dx = cv.Sobel(gray, cv.CV_64F, 1, 0, ksize=3)
 gray_dy = cv.Sobel(gray, cv.CV_64F, 0, 1, ksize=3)

 img_harris = np.zeros(gray.shape)

 for i in range(1, gray.shape[0]-1):
    for j in range(1, gray.shape[1]-1):
        
        dx = np.zeros((3, 3))
        dy = np.zeros((3, 3))
      
        dx = gray_dx[i-1:i+2, j-1:j+2]
        dy = gray_dy[i-1:i+2, j-1:j+2]

        # calculate the sum of squares of derivatives
        dx2 = np.sum(dx*dx)
        dy2 = np.sum(dy*dy)
        dxdy = np.sum(dx*dy)

        # matrix of sum of squares of derivatives
        M = np.array([[dx2, dxdy], [dxdy, dy2]])

        # calculate the harris response
        img_harris[i, j] = np.linalg.det(M) - 0.04 * np.trace(M) * np.trace(M)

 # threshold the harris response
 img_harris[img_harris < 0.01 * img_harris.max()] = 0

 # get the x and y coordinates of the corners
 img_harris_x = np.where(img_harris != 0)[1]
 img_harris_y = np.where(img_harris != 0)[0]

 radius = 3

 img_harris_x = img_harris_x.astype(float)
 img_harris_y = img_harris_y.astype(float)
 
 # get the descriptors of the corners using orb.detect 
 orb = cv.ORB_create()

 kp = [cv.KeyPoint(x, y, radius) for x, y in zip(img_harris_x, img_harris_y)]
 
 kp, des = orb.compute(img, kp)

 return kp, des 

def match_keypoints(des1, des2):
    bf = cv.BFMatcher.create(cv.NORM_HAMMING, crossCheck=True)    # this should be brute force hamming
    matches = bf.match(des1, des2)
    sorted_matches = sorted(matches, key=lambda x: x.distance, reverse=True)
    return sorted_matches

# Print lines that connect matching keypoints across two images
def display_keypoint_matches(img1, img2, kp1, kp2, matches):
    concat_image = np.concatenate((img1, img2), axis=1)
    h, w = img1.shape[:2]
    for m in matches:
        color = (0, 255, 0)
        cv.line(concat_image, (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1])), (int(kp2[m.trainIdx].pt[0] + w), int(kp2[m.trainIdx].pt[1])), color)
    
    plt.imshow(concat_image)
    cv.waitKey(0)
    cv.destroyAllWindows()

# Prune matches based distance attribute
def prune_keypoint_matches_by_distance(matches):
    FACTOR = 10
    pruned_matches = []
    threshold = matches[-1].distance * FACTOR
    for m in matches:
        if m.distance < threshold:
            pruned_matches.append(m)
    return pruned_matches
